In [1]:
#disease_duration   24.01.30
#
#Perform Kruskal–Wallis test to see the difference between ACPA– and ACPA+ RA.
#EAC (Early arthritis cohort = disease duration = 0 )
#ACPA+ RA = 1
#ACPA– RA = 2

#This analysis was done after recieving internal review from the Drs.
#And will be inserted into Table 1

In [2]:
data_file = "../../analysis_addressing_feedback/disease_duration/data/disease_duration.csv"
data_df = read.csv(data_file, sep=",")
data_df$disease_duration[data_df$disease_duration == "EAC"] <- 1
data_df$disease_duration <- as.integer(data_df$disease_duration)

In [6]:
kruskal.test(disease_duration ~ ACPA, data = data_df)


	Kruskal-Wallis rank sum test

data:  disease_duration by ACPA
Kruskal-Wallis chi-squared = 0.02117, df = 1, p-value = 0.8843


In [4]:
#Simple statistics for ACPA+ RA
print (mean(data_df$disease_duration[data_df$ACPA == "1"])) #mean
print (sd(data_df$disease_duration[data_df$ACPA == "1"])) #sd
print (quantile(data_df$disease_duration[data_df$ACPA == "1"], 0.25)) #Q1
print (quantile(data_df$disease_duration[data_df$ACPA == "1"], 0.75)) #Q3
print (min(data_df$disease_duration[data_df$ACPA == "1"])) #min
print (max(data_df$disease_duration[data_df$ACPA == "1"])) #max

[1] 2.9
[1] 2.550013
25% 
  1 
75% 
  4 
[1] 0
[1] 10


In [5]:
#Simple statistics for ACPA– RA
print (mean(data_df$disease_duration[data_df$ACPA == "2"])) #mean
print (sd(data_df$disease_duration[data_df$ACPA == "2"])) #sd
print (quantile(data_df$disease_duration[data_df$ACPA == "2"], 0.25)) #Q1
print (quantile(data_df$disease_duration[data_df$ACPA == "2"], 0.75)) #Q3
print (min(data_df$disease_duration[data_df$ACPA == "2"])) #min
print (max(data_df$disease_duration[data_df$ACPA == "2"])) #max

[1] 2.875
[1] 2.642721
25% 
  1 
75% 
  4 
[1] 0
[1] 10
